Зчитуємо дані

In [3]:
import zipfile
with zipfile.ZipFile("homework.zip","r") as zip_ref:
    zip_ref.extractall()

In [4]:
import pandas as pd
import glob
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score

In [5]:
data_walking = pd.concat(pd.read_csv(f) for f in glob.glob("/content/data/walking/*.csv"))
data_running = pd.concat(pd.read_csv(f) for f in glob.glob("/content/data/running/*.csv"))
data_idle = pd.concat(pd.read_csv(f) for f in glob.glob("/content/data/idle/*.csv"))
data_stairs = pd.concat(pd.read_csv(f) for f in glob.glob("/content/data/stairs/*.csv"))

In [6]:
data_walking['activity'] = 'walking'
data_running['activity'] = 'running'
data_idle['activity'] = 'idle'
data_stairs['activity'] = 'stairs'

data = pd.concat([data_walking, data_running, data_idle, data_stairs])

In [7]:
data.tail(200)

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
10,-0.545878,-3.653552,6.670246,stairs
11,2.844312,-10.486604,-1.580173,stairs
12,2.566584,-10.323798,-1.168370,stairs
13,2.638410,-7.168240,-0.718260,stairs
14,3.907337,-4.984728,-3.821146,stairs
...,...,...,...,...
25,-2.312799,-15.825673,9.706094,stairs
26,-0.627281,-14.772224,-5.382165,stairs
27,2.466028,-6.856993,-4.477157,stairs
28,-4.678270,-6.397307,-3.993528,stairs


In [8]:
data.shape

(193860, 4)

In [19]:
# data['accelerometer_X_mean'] = data.groupby('activity')['accelerometer_X'].transform('mean')
# data['accelerometer_X_std'] = data.groupby('activity')['accelerometer_X'].transform('std')
# data['accelerometer_X_max'] = data.groupby('activity')['accelerometer_X'].transform('max')
# data['accelerometer_X_min'] = data.groupby('activity')['accelerometer_X'].transform('min')
# data['accelerometer_X_range'] = data['accelerometer_X_max'] - data['accelerometer_X_min']

# data['accelerometer_Z_mean'] = data.groupby('activity')['accelerometer_Z'].transform('mean')
# data['accelerometer_Z_std'] = data.groupby('activity')['accelerometer_Z'].transform('std')
# data['accelerometer_Z_max'] = data.groupby('activity')['accelerometer_Z'].transform('max')
# data['accelerometer_Z_min'] = data.groupby('activity')['accelerometer_Z'].transform('min')
# data['accelerometer_Z_range'] = data['accelerometer_Z_max'] - data['accelerometer_Z_min']

# data['accelerometer_Y_mean'] = data.groupby('activity')['accelerometer_Y'].transform('mean')
# data['accelerometer_Y_std'] = data.groupby('activity')['accelerometer_Y'].transform('std')
# data['accelerometer_Y_max'] = data.groupby('activity')['accelerometer_Y'].transform('max')
# data['accelerometer_Y_min'] = data.groupby('activity')['accelerometer_Y'].transform('min')
# data['accelerometer_Y_range'] = data['accelerometer_Y_max'] - data['accelerometer_Y_min']

# data['accelerometer_mean_combined'] = data[['accelerometer_X_mean', 'accelerometer_Y_mean', 'accelerometer_Z_mean']].mean(axis=1)
# data['accelerometer_std_combined'] = data[['accelerometer_X_std', 'accelerometer_Y_std', 'accelerometer_Z_std']].std(axis=1)
# data['accelerometer_max_combined'] = data[['accelerometer_X_max', 'accelerometer_Y_max', 'accelerometer_Z_max']].max(axis=1)
# data['accelerometer_min_combined'] = data[['accelerometer_X_min', 'accelerometer_Y_min', 'accelerometer_Z_min']].min(axis=1)
# data['accelerometer_range_combined'] = data[['accelerometer_X_range', 'accelerometer_Y_range', 'accelerometer_Z_range']].max(axis=1)

Виділяємо ознаки та цільову змінну

In [40]:
X = data[['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']]

y = data['activity']

Розбиваємо датасет на навчальний і тренувальний

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
# Навчання моделі SVM
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train, y_train)

SVC()

In [43]:
# Навчання моделі Випадкового лісу
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)


RandomForestClassifier()

In [44]:
# Прогнозування на тестовому наборі
svm_predictions = svm_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)

In [45]:
# Оцінка продуктивності моделей
svm_accuracy = accuracy_score(y_test, svm_predictions)
rf_accuracy = accuracy_score(y_test, rf_predictions)

print("Точність моделі SVM:", svm_accuracy)
print("Точність моделі Випадкового лісу:", rf_accuracy)

Точність моделі SVM: 0.896084803466419
Точність моделі Випадкового лісу: 0.999664706489219


Точність моделі випадковго лісу вища, а саме навчання відбувається значно швидше